In [64]:
import random
import csv
import numpy as np
def loader(file):
    with open(file) as data:
        data_raw = list(csv.reader(data))
    return data_raw
raw_data = loader("sonar.all-data")
for row in raw_data:
    for i in row[-1]:
        if i == "R" or i == "M":
            key = row.pop(-1)
            if key == "M":
                key = 1
            elif key == "R":
                key = 0
            row.append(key)
for row in raw_data:
    for i in row:
        i == float(i)
dataset = [list(map(float, row)) for row in raw_data]
###### trsize = size of train set (from 0 to 1)
def split(dataset, trsize):
    train_set = []
    test_set = []
    random.shuffle(dataset)
    for i in range(int(len(dataset)*trsize)):
        train_set.append(dataset[i])
    for i in range(int(len(dataset)*(1-trsize))):
        test_set.append(dataset[i])
    return train_set, test_set
        
train_set, test_set = split(dataset, 0.8)
train_set

[[0.0067,
  0.0096,
  0.0024,
  0.0058,
  0.0197,
  0.0618,
  0.0432,
  0.0951,
  0.0836,
  0.118,
  0.0978,
  0.0909,
  0.0656,
  0.0593,
  0.0832,
  0.1297,
  0.2038,
  0.3811,
  0.4451,
  0.5224,
  0.5911,
  0.6566,
  0.6308,
  0.5998,
  0.4958,
  0.5647,
  0.6906,
  0.8513,
  1.0,
  0.9166,
  0.7676,
  0.6177,
  0.5468,
  0.5516,
  0.5463,
  0.5515,
  0.4561,
  0.3466,
  0.3384,
  0.2853,
  0.2502,
  0.1641,
  0.1605,
  0.1491,
  0.1326,
  0.0687,
  0.0602,
  0.0561,
  0.0306,
  0.0154,
  0.0029,
  0.0048,
  0.0023,
  0.002,
  0.004,
  0.0019,
  0.0034,
  0.0034,
  0.0051,
  0.0031,
  0.0],
 [0.0473,
  0.0509,
  0.0819,
  0.1252,
  0.1783,
  0.307,
  0.3008,
  0.2362,
  0.383,
  0.3759,
  0.3021,
  0.2909,
  0.2301,
  0.1411,
  0.1582,
  0.243,
  0.4474,
  0.5964,
  0.6744,
  0.7969,
  0.8319,
  0.7813,
  0.8626,
  0.7369,
  0.4122,
  0.2596,
  0.3392,
  0.3788,
  0.4488,
  0.6281,
  0.7449,
  0.7328,
  0.7704,
  0.787,
  0.6048,
  0.586,
  0.6385,
  0.7279,
  0.6286,
  0.5316,
  0

In [86]:
weights = np.zeros(len(train_set[0])-1)

In [87]:
def predict(train, epoch_n, weights, lrate):
    z = 0
    for i in range(epoch_n):
        predictions = []
        for row in train:
            sum = np.dot(weights, row[:-1])
            predicted = 0.0 if sum<z else 1.0
            predictions.append(predicted)
            for n in range(len(weights)):
                weights[n] += lrate*(row[-1]-predicted)*row[n]
    return predictions
predictions = predict(train_set, 5000, weights, 0.1)
def accuracy(predicted, train):
    score = 0
    actual = [row[-1] for row in train]
    for i in range(len(predicted)):
        if predicted[i] == actual[i]:
            score += 1
    return score/float(len(predicted))*100

def test(test, weights):
    tpred = []
    tactual = [row[-1] for row in test]
    for row in test_set:
        pred = np.dot(row[:-1], weights) 
        tpred.append(0 if pred<0 else 1)
        
    return tpred, tactual
tpred, tactual = test(test_set, weights)

In [88]:
def evaluation(tpred, tacutal):
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for i in range(len(tactual)):
        if tpred[i] == tactual[i]:
            if tpred[i] == 0:
                tn += 1
            else:
                tp += 1
        else:
            if tpred[i] == 0:
                fn += 1
            else:
                fp += 1
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    accuracy = (tp+tn)/(tp+tn+fn+fp)
    f1 = 2*((precision*recall)/(precision+recall))
    return print(f"""Accuracy:  {accuracy}
Precision: {precision}
Recall:    {recall}
F1 Score:  {f1}""")


evaluation(tpred, tactual)    

Accuracy:  0.7804878048780488
Precision: 0.7
Recall:    1.0
F1 Score:  0.8235294117647058


In [89]:
from sklearn.linear_model import Perceptron

# training the sklearn Perceptron
clf = Perceptron(random_state=None, eta0=0.1, shuffle=False, fit_intercept=False, max_iter=5000)
clf.fit([row[:-1] for row in train_set], [row[-1] for row in train_set])
y_predict = clf.predict([row[:-1] for row in test_set])

C:\Users\Irena\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in Perceptron in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [98]:
clf.coef_ == weights

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True]])